In [6]:
import sys
import glob
import json
import cv2
import shutil
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import SimpleITK as sitk
from medpy.io import load
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib.image

In [7]:
columns = ['case','is_use']
positive_case = pd.read_csv('./cases/positive_case.csv', header=None, names=columns)
positive_case = positive_case[positive_case["is_use"] == 1]
positive_case["case"] = "cases/labels/" + positive_case["case"] + ".nii.gz"
positive_case

,case,is_use
0,cases/labels/10005_1000005.nii.gz,1
1,cases/labels/10021_1000021.nii.gz,1
2,cases/labels/10032_1000032.nii.gz,1
3,cases/labels/10043_1000043.nii.gz,1
4,cases/labels/10048_1000048.nii.gz,1
...,...,...
83,cases/labels/11428_1001452.nii.gz,1
85,cases/labels/11444_1001468.nii.gz,1
86,cases/labels/11447_1001471.nii.gz,1
87,cases/labels/11462_1001486.nii.gz,1


In [8]:
file_list = sorted(glob('cases/labels/10021_1000021.nii.gz'))

In [9]:
file_list = []
for i in range(len(positive_case)):
    file_list.append(positive_case.iloc[i, 0])

In [10]:
def load_image(path_nii):
    image_data, image_header = load(path_nii)
    image_sitk = image_header.get_sitkimage()
    image_direction = image_header.get_direction()
    image_offset = image_header.get_offset()
    image_voxel_spacing = image_header.get_voxel_spacing()
    return image_data, image_sitk, image_header

In [15]:
# export only dwi file and label

threshold = 0

for f_lbl in file_list:    
    lbl, _, _ = load_image(f_lbl)
    f_img = f_lbl.replace('labels','images').replace('.nii.gz','')
    
    print("working in " + f_img)
    
    im2, _, _ = load_image(f_img + '_0002.nii.gz') # dwi - hbv
    assert (lbl.shape[2] == im2.shape[2])
    
    for index in range(1, lbl.shape[2]):
        """ Save ADC HBV T2W Label from numpy array """
        matplotlib.image.imsave( './export_dwi/' + f_img.replace("cases/images/", "") + '_dwi_' + str(index).rjust(2, '0') +'.jpg', im2[:,:,index], cmap='jet')
#         convert_grayscale = np.where(lbl[:,:,index] > threshold, True, False)
#         matplotlib.image.imsave( './export_mask/' + f_lbl.replace("cases/labels/", "").replace('.nii.gz',"") + "_dwi_" + str(index).rjust(2, '0') + "_mask" + '.gif', convert_grayscale, cmap='gray')
        #break
    #break
print("done")

working in cases/images/10005_1000005
working in cases/images/10021_1000021
working in cases/images/10032_1000032
working in cases/images/10043_1000043
working in cases/images/10048_1000048
working in cases/images/10078_1000078
working in cases/images/10079_1000079
working in cases/images/10085_1000085
working in cases/images/10117_1000117
working in cases/images/10118_1000118
working in cases/images/10211_1000215
working in cases/images/10216_1000220
working in cases/images/10268_1000272
working in cases/images/10322_1000328
working in cases/images/10339_1000345
working in cases/images/10340_1000346
working in cases/images/10368_1000374
working in cases/images/10434_1000442
working in cases/images/10451_1000459
working in cases/images/10463_1000471
working in cases/images/10503_1000512
working in cases/images/10523_1000533
working in cases/images/10558_1000570
working in cases/images/10570_1000582
working in cases/images/10607_1000621
working in cases/images/10635_1000651
working in c

In [17]:
np.unique(convert_grayscale)

array([False])

In [22]:
!rm -f /media/mpmri/mpmri/Pytorch-UNet/data/imgs/*
!cp /media/mpmri/mpmri/code_isup/export_dwi/*dwi* /media/mpmri/mpmri/Pytorch-UNet/data/imgs/

In [23]:
!rm -f /media/mpmri/mpmri/Pytorch-UNet/data/masks/*
!cp /media/mpmri/mpmri/code_isup/export_mask/*_mask* /media/mpmri/mpmri/Pytorch-UNet/data/masks